In [1]:
import os
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.exceptions import NotFittedError

import typing
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.exceptions import NotFittedError

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
data = pd.read_csv('../data/train.csv', low_memory=False)
test = pd.read_csv('../data/test.csv', low_memory=False)

In [4]:
TARGET = 'per_square_meter_price'
# признаки (или набор признаков), для которых применяем smoothed target encoding
CATEGORICAL_STE_FEATURES = ['region', 'city', 'realty_type']

# признаки, для которых применяем one hot encoding
CATEGORICAL_ONE_FEATURES = []

# численные признаки
NUM_FEATURES = ['lat', 'lng', 'osm_amenity_points_in_0.001',
       'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075',
       'osm_amenity_points_in_0.01', 'osm_building_points_in_0.001',
       'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
       'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
       'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075',
       'osm_catering_points_in_0.01', 'osm_city_closest_dist',
      'osm_city_nearest_population',
       'osm_crossing_closest_dist', 'osm_crossing_points_in_0.001',
       'osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075',
       'osm_crossing_points_in_0.01', 'osm_culture_points_in_0.001',
       'osm_culture_points_in_0.005', 'osm_culture_points_in_0.0075',
       'osm_culture_points_in_0.01', 'osm_finance_points_in_0.001',
       'osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075',
       'osm_finance_points_in_0.01', 'osm_healthcare_points_in_0.005',
       'osm_healthcare_points_in_0.0075', 'osm_healthcare_points_in_0.01',
       'osm_historic_points_in_0.005', 'osm_historic_points_in_0.0075',
       'osm_historic_points_in_0.01', 'osm_hotels_points_in_0.005',
       'osm_hotels_points_in_0.0075', 'osm_hotels_points_in_0.01',
       'osm_leisure_points_in_0.005', 'osm_leisure_points_in_0.0075',
       'osm_leisure_points_in_0.01', 'osm_offices_points_in_0.001',
       'osm_offices_points_in_0.005', 'osm_offices_points_in_0.0075',
       'osm_offices_points_in_0.01', 'osm_shops_points_in_0.001',
       'osm_shops_points_in_0.005', 'osm_shops_points_in_0.0075',
       'osm_shops_points_in_0.01', 'osm_subway_closest_dist',
       'osm_train_stop_closest_dist', 'osm_train_stop_points_in_0.005',
       'osm_train_stop_points_in_0.0075', 'osm_train_stop_points_in_0.01',
       'osm_transport_stop_closest_dist', 'osm_transport_stop_points_in_0.005',
       'osm_transport_stop_points_in_0.0075',
       'osm_transport_stop_points_in_0.01',
       'reform_count_of_houses_1000', 'reform_count_of_houses_500',
       'reform_house_population_1000', 'reform_house_population_500',
       'reform_mean_floor_count_1000', 'reform_mean_floor_count_500',
       'reform_mean_year_building_1000', 'reform_mean_year_building_500','total_square']

MODEL_PARAMS = dict(
            n_estimators=2000,
            learning_rate=0.01,
            reg_alpha=1,
            num_leaves=40,
            min_child_samples=5,
            importance_type="gain",
            n_jobs=1,
            random_state=563,
        )

In [5]:
data['floor'] = data['floor'].fillna(50) #средняя цена для этажа
test['floor'] = test['floor'].fillna(50) #средняя цена для этажа

In [6]:
pd.unique(data['floor']), pd.unique(test['floor'])

(array([50, '3.0', '4.0', '-1.0', '1.0', '14.0', '2.0', '8.0', '-2.0',
        '6.0', '10.0', '5.0', '16.0', '19.0', '7.0', '9.0', '58.0', '24.0',
        '18.0', '26.0', '17.0', '48.0', '11.0', '-3.0', '15.0', '22.0',
        '60.0', '12.0', '21.0', '35.0', '28.0', '38.0', '39.0', '13.0',
        '81.0', '44.0', '82.0', '25.0', '45.0', '47.0', '23.0', '37.0',
        '29.0', '113.0', '78.0', '42.0', '69.0', '27.0', '46.0', '53.0',
        '80.0', '70.0', '76.0', '64.0', '30.0', '73.0', '77.0', '52.0',
        '67.0', '65.0', '20.0', '40.0', '49.0', '75.0', '93.0', '94.0',
        '91.0', '72.0', '79.0', '84.0', '92.0', '33.0', '66.0', '90.0',
        '31.0', '36.0', '61.0', '71.0', '68.0', '51.0', '97.0', '43.0',
        '95.0', '85.0', '50.0', '0.0', '62.0', '54.0', '74.0', '57.0',
        '41.0', '34.0', '59.0', '56.0', '123.0', '55.0', '83.0', '1',
        'подвал, 1', '2', 'подвал', 'цоколь, 1', '1,2,антресоль', 'цоколь',
        '4', '5', 'тех.этаж (6)', '3', 'Подвал', 'Цоколь', 

In [7]:
bad_floor_dict = {'nan': 50, "подвал, 1": -1, 'подвал': -1, 'цоколь, 1': 1, 
                  '1,2,антресоль': 4, 'цоколь': 1, 'тех.этаж (6)': 6, 
                  'Подвал': -1, 'Цоколь': 1, 'фактически на уровне 1 этажа': 1,
                  '1,2,3': 2, '1, подвал': -1, '1,2,3,4': 2, '1,2': 1, 
                  '1,2,3,4,5': 3, '5, мансарда': 5, '1-й, подвал': -1, 
                  '1, подвал, антресоль': -1, 'мезонин': 4, 'подвал, 1-3': -2, 
                  '1 (Цокольный этаж)': 1, '3, Мансарда (4 эт)': 4, 
                  'подвал,1': -1, '1, антресоль': 1, '1-3': 2, 'мансарда (4эт)': 4,
                  '1, 2.': 2, 'подвал , 1 ': -1, '1, 2': 2, 'подвал, 1,2,3': -2,
                  '1 + подвал (без отделки)': -1, 'мансарда': 4, '2,3': 2, 
                  "4, 5": 4, '1-й, 2-й': 2, '1 этаж, подвал': -1, '1, цоколь': 1, 
                  'подвал, 1-7, техэтаж': 4, '3 (антресоль)': 3, '1, 2, 3': 2,
                  'Цоколь, 1,2(мансарда)': 2, 'подвал, 3. 4 этаж': 2, 
                  'подвал, 1-4 этаж': 2, 'подва, 1.2 этаж': 1, '2, 3': 2, '1.2': 2,
                  '7,8': 7, '1 этаж': 1, '1-й': 1,  '3 этаж': 3, '4 этаж': 4, 
                  '5 этаж': 5, 'подвал,1,2,3,4,5': 2, 'подвал, цоколь, 1 этаж': 1,
                  '3, мансарда': 3, 'цоколь, 1, 2,3,4,5,6' : 3, ' 1, 2, Антресоль' : 2, 
                  '3 этаж, мансарда (4 этаж)' : 3.5, 'цокольный' : 1, '1,2 ' : 1.5, 
                  '3,4' : 3.5, 'подвал, 1 и 4 этаж' : 1, '5(мансарда)': 5,
                  'технический этаж,5,6' : 5.5, ' 1-2, подвальный' : 1, '1, 2, 3, мансардный' : 2,
                  'подвал, 1, 2, 3' : 2, '1,2,3, антресоль, технический этаж' : 2.5, '3, 4' : 3.5,
                  '1-3 этажи, цоколь (188,4 кв.м), подвал (104 кв.м)' : 2.5, '1,2,3,4, подвал' : 2, 
                  '2-й' : 2, '1, 2 этаж' : 1.5, 'подвал, 1, 2' : 1, '1-7' : 4.5, '1 (по док-м цоколь)' : 1,
                  '1,2,подвал ' : 1, 'подвал, 2' : 1, 'подвал,1,2,3' : 1.5, '1,2,3 этаж, подвал' : 1.5,
                  '2, 3, 4, тех.этаж' : 3, 'цокольный, 1,2' : 1, 'Техническое подполье' : 1}
def clear_unique(data):
    for ind, row in data.iterrows():
        if row['floor'] in bad_floor_dict:
          data.loc[ind, 'floor'] = bad_floor_dict.get(row['floor'], 50)
    return data

In [8]:
data = clear_unique(data)
test = clear_unique(test)

In [9]:
pd.unique(data['floor']), pd.unique(test['floor'])

(array([50, '3.0', '4.0', '-1.0', '1.0', '14.0', '2.0', '8.0', '-2.0',
        '6.0', '10.0', '5.0', '16.0', '19.0', '7.0', '9.0', '58.0', '24.0',
        '18.0', '26.0', '17.0', '48.0', '11.0', '-3.0', '15.0', '22.0',
        '60.0', '12.0', '21.0', '35.0', '28.0', '38.0', '39.0', '13.0',
        '81.0', '44.0', '82.0', '25.0', '45.0', '47.0', '23.0', '37.0',
        '29.0', '113.0', '78.0', '42.0', '69.0', '27.0', '46.0', '53.0',
        '80.0', '70.0', '76.0', '64.0', '30.0', '73.0', '77.0', '52.0',
        '67.0', '65.0', '20.0', '40.0', '49.0', '75.0', '93.0', '94.0',
        '91.0', '72.0', '79.0', '84.0', '92.0', '33.0', '66.0', '90.0',
        '31.0', '36.0', '61.0', '71.0', '68.0', '51.0', '97.0', '43.0',
        '95.0', '85.0', '50.0', '0.0', '62.0', '54.0', '74.0', '57.0',
        '41.0', '34.0', '59.0', '56.0', '123.0', '55.0', '83.0', '1', -1,
        '2', 1, 4, '4', '5', 6, '3', '10', '6', 2, 3, 5, '12', '15', '13',
        -2, '8', '7', '9', '18', '-1', '11', '36', 7, '2

In [10]:
data['floor'] = pd.to_numeric(data['floor'])
test['floor'] = pd.to_numeric(test['floor'])

In [11]:
list_for_nan = []
for column in data:
    if data[column].isna().sum() > 0:
      list_for_nan.append(column)
      print(column, data[column].isna().sum())
list_for_nan.remove('street')

list_for_nan = []
for column in test:
    if test[column].isna().sum() > 0:
      list_for_nan.append(column)
      print(column, test[column].isna().sum())
list_for_nan.remove('street')

osm_city_nearest_population 55
reform_house_population_1000 14596
reform_house_population_500 27234
reform_mean_floor_count_1000 16708
reform_mean_floor_count_500 30168
reform_mean_year_building_1000 16239
reform_mean_year_building_500 29637
street 1606
reform_house_population_1000 35
reform_house_population_500 48
reform_mean_floor_count_1000 35
reform_mean_floor_count_500 48
reform_mean_year_building_1000 35
reform_mean_year_building_500 48
street 6


In [12]:
p = data
p.fillna(0)
for i in list_for_nan:
  data[i].fillna(pd.to_numeric(p[i]).mean())

p = test
p.fillna(0)
for i in list_for_nan:
  test[i].fillna(pd.to_numeric(p[i]).mean())

In [13]:
scl = StandardScaler()
num_features = data[NUM_FEATURES]
scl.fit(num_features)

num_features = scl.transform(test[NUM_FEATURES])
test[NUM_FEATURES] = num_features

num_features = scl.transform(data[NUM_FEATURES])
data[NUM_FEATURES] = num_features

In [14]:
data = data.drop('id', 1)
data = data.drop('date', 1)
data = data.drop("street", 1)
data = data.drop("city", 1)
data = data.drop("region", 1)

test = test.drop('id', 1)
test = test.drop('date', 1)
test = test.drop("street", 1)
test = test.drop("city", 1)
test = test.drop("region", 1)

/tmp/ipykernel_3118/831240988.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop('id', 1)
/tmp/ipykernel_3118/831240988.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop('date', 1)
/tmp/ipykernel_3118/831240988.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop("street", 1)
/tmp/ipykernel_3118/831240988.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop("city", 1)
/tmp/ipykernel_3118/831240988.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop("region", 1)
/tmp/ipykerne

In [15]:
from sklearn.preprocessing import OneHotEncoder

CATEGORICAL_OHE_FEATURES = ["osm_city_nearest_name"]
def get_one_hot(data):
  for column in CATEGORICAL_OHE_FEATURES:
      tempdf = pd.get_dummies(data[column], prefix=column)
      data = pd.merge(
          left=data,
          right=tempdf,
          left_index=True,
          right_index=True,
      )
      data = data.drop(columns=column)
  return data

In [16]:
data = get_one_hot(data)
test = get_one_hot(test)

In [17]:
data.head() 

,floor,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,...,osm_city_nearest_name_Черкесск,osm_city_nearest_name_Чита,osm_city_nearest_name_Шахты,osm_city_nearest_name_Щёлково,osm_city_nearest_name_Электросталь,osm_city_nearest_name_Элиста,osm_city_nearest_name_Энгельс,osm_city_nearest_name_Ярославль,osm_city_nearest_name_绥芬河市,osm_city_nearest_name_饶河县
0,50.0,0.855954,0.500408,0.307182,-0.405401,-0.442959,-0.471795,-0.095757,-0.129142,-0.13826,...,0,0,0,0,0,0,0,0,0,0
1,50.0,0.285042,-0.482247,0.069225,-0.311580,-0.423946,-0.431166,-0.095757,-0.129142,-0.13826,...,0,0,0,0,0,0,0,0,0,0
2,50.0,0.766672,-0.464254,-0.406688,-0.198996,-0.138756,-0.030678,-0.095757,-0.129142,-0.07070,...,0,0,0,0,0,0,0,1,0,0
3,50.0,-0.109992,2.308360,-0.644645,-0.761919,-0.728149,-0.651724,-0.095757,-0.129142,-0.13826,...,0,0,0,0,0,0,0,0,0,0
4,50.0,0.338816,-0.602914,-0.406688,-0.330344,-0.167275,0.114427,-0.095757,0.016666,-0.07070,...,0,0,0,0,0,0,0,0,0,0


In [18]:
for missing_coloumn in data.columns.difference(test.columns):
    if missing_coloumn != TARGET:
        test[missing_coloumn] = 0

In [19]:
data.to_csv('../data/data_processed.csv')
test.to_csv('../data/test_processed.csv')

In [20]:
data.columns.difference(test.columns), test.columns.difference(data.columns)

(Index(['per_square_meter_price'], dtype='object'), Index([], dtype='object'))